# The Battle of Neighborhoods

### Instalando dependencias

In [5]:
!conda install -c conda-forge geopy --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

In [3]:
!conda install -c conda-forge folium --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    folium-0.10.1              |             py_0          59 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTAL

### Import packages

In [4]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm # Matplotlib and associated plotting modules
import matplotlib.colors as colors
from sklearn.cluster import KMeans # import k-means from clustering stage
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Download and Explore Dataset

#### Dataset 1. Coordinates of the neighborhoods
Obtaining coordinate data for New York neighborhoods

In [5]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [6]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Exploring the data structure and obtaining the necessary data

In [8]:
newyork_data['features'][0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [9]:
neighborhoods_data = newyork_data['features']

In [10]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

"neighborhoods" will contain the coordinate data

In [11]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Loading the source data to "neighborhoods" target dataframe

In [12]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [13]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [14]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


The dataframe "neighborhoods" has 5 boroughs and 306 neighborhoods. According to the data requirements, the neighborhoods of Manhattan will be considered, in this way we obtain the data set of Manhattan

In [15]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


#### Dataset 2. Foursquare API to get and explore Manhattan venues

In [16]:
CLIENT_ID = 'N1QOETOCEDFTSUI2GA0OFHVDAWS5XXHDJ4JWJENBYXVDY2ZI' # your Foursquare ID
CLIENT_SECRET = 'ZAJVVTHMNHXTDHESFDHMXPM3UW5UELOY4IFPLVNQ2YH0RMXV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: N1QOETOCEDFTSUI2GA0OFHVDAWS5XXHDJ4JWJENBYXVDY2ZI
CLIENT_SECRET:ZAJVVTHMNHXTDHESFDHMXPM3UW5UELOY4IFPLVNQ2YH0RMXV


Let's getting 100 places 500 meters around for each neighborhood

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


"manhattan_venues" dataset contains the data of the neighborhoods and their places located 500 meters around with their respective coordinates. Exploring the resulting dataset "manhattan_venues"

In [22]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3158, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop
4,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop


### Analysis and Preparing Dataset

For our analysis we must prepare the dataset for our model, therefore the categorical variable "venues" we will apply the one hot encoding process

In [23]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Video Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [24]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Video Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Battery Park City,0.000000,0.00,0.00,0.000000,0.014493,0.00,0.000000,0.000000,0.00,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.028986,0.000000,0.014493,0.000000
1,Carnegie Hill,0.000000,0.00,0.00,0.000000,0.011364,0.00,0.000000,0.000000,0.00,...,0.00,0.011364,0.000000,0.000000,0.000000,0.011364,0.034091,0.000000,0.011364,0.034091
2,Central Harlem,0.000000,0.00,0.00,0.044444,0.044444,0.00,0.000000,0.000000,0.00,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Chelsea,0.000000,0.00,0.00,0.000000,0.040000,0.00,0.000000,0.000000,0.00,...,0.00,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000,0.000000,0.010000
4,Chinatown,0.000000,0.00,0.00,0.000000,0.040000,0.00,0.000000,0.000000,0.00,...,0.00,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000
5,Civic Center,0.000000,0.00,0.00,0.000000,0.030000,0.01,0.000000,0.000000,0.00,...,0.00,0.000000,0.000000,0.000000,0.010000,0.010000,0.020000,0.000000,0.000000,0.030000
6,Clinton,0.000000,0.00,0.00,0.000000,0.030000,0.00,0.000000,0.000000,0.00,...,0.00,0.000000,0.000000,0.000000,0.000000,0.010000,0.030000,0.000000,0.000000,0.000000
7,East Harlem,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,East Village,0.000000,0.00,0.00,0.000000,0.020000,0.00,0.000000,0.010000,0.01,...,0.00,0.020000,0.000000,0.000000,0.000000,0.040000,0.010000,0.000000,0.000000,0.000000
9,Financial District,0.000000,0.00,0.00,0.000000,0.030000,0.00,0.000000,0.000000,0.00,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.010000,0.010000


Look at each neighborhood along with the top 5 most common venues

In [25]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
           venue  freq
0           Park  0.09
1          Hotel  0.07
2            Gym  0.06
3    Coffee Shop  0.04
4  Memorial Site  0.04


----Carnegie Hill----
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.06
2  Japanese Restaurant  0.03
3               Bakery  0.03
4   Italian Restaurant  0.03


----Central Harlem----
                  venue  freq
0    Chinese Restaurant  0.04
1  Gym / Fitness Center  0.04
2    Seafood Restaurant  0.04
3     French Restaurant  0.04
4   Fried Chicken Joint  0.04


----Chelsea----
                 venue  freq
0          Coffee Shop  0.09
1          Art Gallery  0.05
2  American Restaurant  0.04
3   Italian Restaurant  0.03
4               Bakery  0.03


----Chinatown----
                 venue  freq
0   Chinese Restaurant  0.09
1         Cocktail Bar  0.05
2  American Restaurant  0.04
3               Bakery  0.04
4                  Spa  0.03


----Civic Center----
               venue  fr

Finally we remove the "Neighborhood" column to get our input dataset to the model.

In [26]:
manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)
manhattan_grouped_clustering.head()

,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,...,Video Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0.0,0.0,0.0,0.000000,0.014493,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.028986,0.0,0.014493,0.000000
1,0.0,0.0,0.0,0.000000,0.011364,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.011364,0.0,0.0,0.0,0.011364,0.034091,0.0,0.011364,0.034091
2,0.0,0.0,0.0,0.044444,0.044444,0.0,0.0,0.0,0.0,0.044444,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000
3,0.0,0.0,0.0,0.000000,0.040000,0.0,0.0,0.0,0.0,0.050000,...,0.0,0.000000,0.0,0.0,0.0,0.010000,0.010000,0.0,0.000000,0.010000
4,0.0,0.0,0.0,0.000000,0.040000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.030000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.010000


### Modeling

In the business understanding the analytical approach of the data was determined, this is the K means clustering algorithm

In [27]:
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 2, 3, 2, 3, 2, 2, 3, 2, 1], dtype=int32)

### Results

The generated labels will be associated to the corresponding neighborhood to observe the clusters. For this we will create a DataFrame with the neighborhoods and the 10 most frequent places that we can find around

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Hotel,Gym,Coffee Shop,Memorial Site,Shopping Mall,Gourmet Shop,Wine Shop,Mexican Restaurant,Food Court
1,Carnegie Hill,Coffee Shop,Café,Yoga Studio,Italian Restaurant,Japanese Restaurant,Bakery,Gym,Gym / Fitness Center,Pizza Place,Cosmetics Shop
2,Central Harlem,Gym / Fitness Center,French Restaurant,Fried Chicken Joint,African Restaurant,American Restaurant,Bar,Seafood Restaurant,Chinese Restaurant,Art Gallery,Bookstore
3,Chelsea,Coffee Shop,Art Gallery,American Restaurant,Bakery,Italian Restaurant,Gym / Fitness Center,Cocktail Bar,Pizza Place,Pet Store,Nightclub
4,Chinatown,Chinese Restaurant,Cocktail Bar,American Restaurant,Bakery,Spa,Dessert Shop,Optical Shop,Hotpot Restaurant,Vietnamese Restaurant,Salon / Barbershop


Labeling neighborhoods (Cluster Labels)

In [30]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Sandwich Place,Coffee Shop,Gym,Ice Cream Shop,Department Store,Pharmacy,Diner,Discount Store,Donut Shop,Bank
1,Manhattan,Chinatown,40.715618,-73.994279,3,Chinese Restaurant,Cocktail Bar,American Restaurant,Bakery,Spa,Dessert Shop,Optical Shop,Hotpot Restaurant,Vietnamese Restaurant,Salon / Barbershop
2,Manhattan,Washington Heights,40.851903,-73.936900,3,Café,Bakery,Grocery Store,Chinese Restaurant,Mobile Phone Shop,Deli / Bodega,Coffee Shop,Supermarket,Supplement Shop,Tapas Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,3,Mexican Restaurant,Pizza Place,Restaurant,Café,Lounge,Frozen Yogurt Shop,Bakery,Park,Chinese Restaurant,Deli / Bodega
4,Manhattan,Hamilton Heights,40.823604,-73.949688,3,Pizza Place,Café,Coffee Shop,Deli / Bodega,Mexican Restaurant,Bakery,Sushi Restaurant,Cocktail Bar,Sandwich Place,Yoga Studio


Let's visualize the resulting clusters in the map, therefore we get the geographical coordinates of Manhattan.

In [32]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [33]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Finally, let's visualize the resulting clusters

### Cluster 1

In [34]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Midtown South,Korean Restaurant,Hotel,Japanese Restaurant,Dessert Shop,Burger Joint,Gym / Fitness Center,Clothing Store,Bar,American Restaurant,Coffee Shop


### Cluster 2

In [36]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Sandwich Place,Coffee Shop,Gym,Ice Cream Shop,Department Store,Pharmacy,Diner,Discount Store,Donut Shop,Bank
11,Roosevelt Island,Park,Pizza Place,Sandwich Place,Gym / Fitness Center,Gym,Coffee Shop,Liquor Store,Outdoors & Recreation,Dry Cleaner,School
15,Midtown,Coffee Shop,Hotel,Sporting Goods Shop,Clothing Store,Steakhouse,Spa,Bakery,Café,Sandwich Place,Theater
16,Murray Hill,Sandwich Place,Hotel,Coffee Shop,Sushi Restaurant,Bar,Japanese Restaurant,Burger Joint,Gym / Fitness Center,American Restaurant,Cuban Restaurant
26,Morningside Heights,Park,Bookstore,American Restaurant,Coffee Shop,Deli / Bodega,Burger Joint,Pub,Farmers Market,Mediterranean Restaurant,Supermarket
28,Battery Park City,Park,Hotel,Gym,Coffee Shop,Memorial Site,Shopping Mall,Gourmet Shop,Wine Shop,Mexican Restaurant,Food Court
29,Financial District,Coffee Shop,Pizza Place,Hotel,Mexican Restaurant,Cocktail Bar,Bar,Park,Gym,Gym / Fitness Center,Juice Bar


### Cluster 3

In [37]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Manhattanville,Coffee Shop,Seafood Restaurant,Liquor Store,Italian Restaurant,Deli / Bodega,Park,Mexican Restaurant,Bus Stop,Bus Station,Spanish Restaurant
8,Upper East Side,Italian Restaurant,Coffee Shop,Bakery,Exhibit,Spa,French Restaurant,Juice Bar,Yoga Studio,Gym / Fitness Center,Wine Shop
9,Yorkville,Italian Restaurant,Coffee Shop,Gym,Bar,Deli / Bodega,Pizza Place,Sushi Restaurant,Diner,Wine Shop,Mexican Restaurant
10,Lenox Hill,Italian Restaurant,Coffee Shop,Sushi Restaurant,Pizza Place,Cocktail Bar,Café,Gym,Gym / Fitness Center,Burger Joint,Turkish Restaurant
12,Upper West Side,Italian Restaurant,Coffee Shop,Bar,Bakery,Pizza Place,Ice Cream Shop,Café,Seafood Restaurant,Shoe Store,Bookstore
13,Lincoln Square,Plaza,Café,Gym / Fitness Center,Performing Arts Venue,Theater,Concert Hall,Italian Restaurant,Wine Shop,American Restaurant,Indie Movie Theater
14,Clinton,Theater,Gym / Fitness Center,Coffee Shop,Italian Restaurant,Spa,Thai Restaurant,Sandwich Place,Gym,Hotel,Cocktail Bar
17,Chelsea,Coffee Shop,Art Gallery,American Restaurant,Bakery,Italian Restaurant,Gym / Fitness Center,Cocktail Bar,Pizza Place,Pet Store,Nightclub
18,Greenwich Village,Italian Restaurant,Café,Coffee Shop,Gym,French Restaurant,Dessert Shop,Sushi Restaurant,Gourmet Shop,Pilates Studio,Cocktail Bar
19,East Village,Bar,Pizza Place,Wine Bar,Coffee Shop,Mexican Restaurant,Speakeasy,Cocktail Bar,Ramen Restaurant,Ice Cream Shop,Korean Restaurant


### Cluster 4

In [38]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Cocktail Bar,American Restaurant,Bakery,Spa,Dessert Shop,Optical Shop,Hotpot Restaurant,Vietnamese Restaurant,Salon / Barbershop
2,Washington Heights,Café,Bakery,Grocery Store,Chinese Restaurant,Mobile Phone Shop,Deli / Bodega,Coffee Shop,Supermarket,Supplement Shop,Tapas Restaurant
3,Inwood,Mexican Restaurant,Pizza Place,Restaurant,Café,Lounge,Frozen Yogurt Shop,Bakery,Park,Chinese Restaurant,Deli / Bodega
4,Hamilton Heights,Pizza Place,Café,Coffee Shop,Deli / Bodega,Mexican Restaurant,Bakery,Sushi Restaurant,Cocktail Bar,Sandwich Place,Yoga Studio
6,Central Harlem,Gym / Fitness Center,French Restaurant,Fried Chicken Joint,African Restaurant,American Restaurant,Bar,Seafood Restaurant,Chinese Restaurant,Art Gallery,Bookstore
7,East Harlem,Mexican Restaurant,Bakery,Thai Restaurant,Latin American Restaurant,Deli / Bodega,Gas Station,Cocktail Bar,Liquor Store,Taco Place,Sandwich Place
20,Lower East Side,Chinese Restaurant,Art Gallery,Japanese Restaurant,Bakery,Pizza Place,Café,Ramen Restaurant,Coffee Shop,Yoga Studio,Bubble Tea Shop
22,Little Italy,Bakery,Café,Chinese Restaurant,Bubble Tea Shop,Mediterranean Restaurant,Hotel,Hotpot Restaurant,Italian Restaurant,Thai Restaurant,Coffee Shop
36,Tudor City,Park,Café,Deli / Bodega,Mexican Restaurant,Diner,Greek Restaurant,Garden,Thai Restaurant,Gym / Fitness Center,Sushi Restaurant


### Cluster 5

In [39]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Stuyvesant Town,Boat or Ferry,Park,Bar,Pet Service,German Restaurant,Baseball Field,Fountain,Harbor / Marina,Cocktail Bar,Coffee Shop
